In [1]:
# I want to download data from Yahoo
# Run a MACD long and short 


In [2]:
import sys
sys.path

['E:\\Python310\\python310.zip',
 'E:\\Python310\\DLLs',
 'E:\\Python310\\lib',
 'E:\\Python310',
 'D:\\Projects\\Python\\nautrade\\.venv',
 '',
 'D:\\Projects\\Python\\nautrade\\.venv\\lib\\site-packages']

In [3]:
import yfinance

In [4]:
import datetime as dt

from nautilus_trader.config import StrategyConfig

from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence

from nautilus_trader.trading.strategy import Strategy

from nautilus_trader.model.data import BarType
from nautilus_trader.model.data import Bar
from nautilus_trader.model.position import Position
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide

class MACDStrategyConfig(StrategyConfig):

    instrument_id: InstrumentId
    bar_type_1day: BarType
    fast_period: int = 4
    slow_period: int = 8
    trading_side = 10_000
    enter_threshold: float = 0.00010

# Create a subclass of Strategy class
class MACDStrategy(Strategy):
    def __init__(self, config: MACDStrategyConfig):
        super().__init__()
        self.bar_type_1day = config.bar_type_1day

        self.macd = MovingAverageConvergenceDivergence(
            fast_period = config.fast_period,
            slow_period = config.slow_period,
            price_type = PriceType.MID
        )
        self.instrument_id = config.instrument_id
        self.trade_size = Quantity.from_int(config.trading_side)
        
        self.position: Position | None = None
        self.enter_threshold = config.enter_threshold
        
        self.count_processed_bars = 0
        
        self.start_time = None
        self.end_time = None

    def on_start(self):
        self.start_time = dt.datetime.now()

        self.subscribe_bars(self.bar_type_1day)

        self.log.info(f"My MACD strategy started at {self.start_time}")

    def on_bar(self, bar: Bar):
    
        self.count_processed_bars += 1

        self.macd.handle_bar(bar)
        if not self.macd.initialized:
            return

        self.check_for_entry()
        self.check_for_exit()

    def check_for_entry(self):
        if self.macd.value >= self.enter_threshold:
            # If already in Long position do nothing
            if self.position and self.position.side == PositionSide.LONG:
                return
            # Else make order
            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.BUY,
                quantity = self.trade_size
            )
            self.submit_order(order)

        elif self.macd.value < -self.enter_threshold:
            # If already in Short poisition do nothing
            if self.position and self.position.side == PositionSide.SHORT:
                return
            # Else make order
            order = self.order_factory.market(
                instrument_id = self.instrument_id,
                order_side = OrderSide.SELL,
                quantity = self.trade_size
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If we are in a Short position, exit when MACD value is positive
        if self.macd.value >= 0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        
        # If we are in a Long position, exit when MACD value is negative
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)
        
    
    def on_end(self):
        
        self.end_time = dt.datetime.now()
        self.close_all_positions(instrument_id = self.config.instrument_id)
        self.unsubscribe_bars()
        
        self.log.info(f"My MACD strategy finnished at {self.end_time}")
        self.log.info(f"Total count of 1 day bars: {self.count_processed_bars} ")
    

In [5]:
from decimal import Decimal
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.identifiers import Symbol
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.instruments import Equity
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.objects import Price, Quantity


instrument_id = InstrumentId(
        symbol=Symbol("NVDA"),
        venue=Venue("NASDAQ"),
    )


# Create the NVIDIA equity instrument
NVDA_STOCKS_INSTRUMENT = Equity(
    instrument_id=instrument_id,
    raw_symbol=Symbol("NVDA"),
    currency=USD,
    price_precision=2,  # Prices to 2 decimal places
    price_increment=Price.from_str("0.01"),
    lot_size=Quantity.from_int(1),  # Trading in whole shares
    isin="US67066G1040",  # NVIDIA's ISIN identifier
    ts_event=0,  # Timestamp when the instrument was created/updated
    ts_init=0,   # Timestamp when this object was initialized
)



In [6]:
NVDA_STOCKS_1DAY_BARTYPE = BarType.from_str(
    f"{NVDA_STOCKS_INSTRUMENT.id}-1-DAY-LAST-EXTERNAL"
)



In [7]:
from nautilus_trader.backtest.engine import BacktestEngine

from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.config import ImportableStrategyConfig

from nautilus_trader.model.identifiers import TraderId
from nautilus_trader.model.data import Bar

engine_config = BacktestEngineConfig(
    trader_id = TraderId("BACKTEST-NVDA1DAY-001"),
    strategies = [
        ImportableStrategyConfig(
        strategy_path = '__main__:MACDStrategy',
        config_path = '__main__:MACDStrategyConfig',
        config = {
            "instrument_id": instrument_id,
            "bar_type_1day": NVDA_STOCKS_1DAY_BARTYPE,
            "fast_period": 10,
            "slow_period": 20,
            "enter_threshold": 0.00001
        }
        )
    ],
    logging = LoggingConfig(log_level = "DEBUG"),
    
)

engine = BacktestEngine(config = engine_config)

In [8]:
from nautilus_trader.model.identifiers import Venue
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.objects import Money

engine.add_venue(
        venue=Venue("NASDAQ"),
        oms_type=OmsType.NETTING,  # Order Management System type
        account_type=AccountType.MARGIN,  # Type of trading account
        starting_balances=[Money(1_000_000, USD)],  # Initial account balance
        base_currency=USD,  # Base currency for account
        default_leverage=Decimal(1),  # No leverage used for account
    )


In [9]:
ydata_df = yfinance.download(tickers=['NVDA'], start='2000-01-01', end='2025-01-01')

# Turn yahoo data into Nautilus Data type

ydata_df.head()

C:\Users\juncai\AppData\Local\Temp\ipykernel_13964\343867085.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ydata_df = yfinance.download(tickers=['NVDA'], start='2000-01-01', end='2025-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2000-01-03,0.089420,0.090972,0.084287,0.090256,300912000
2000-01-04,0.087033,0.088107,0.082615,0.087868,300480000
2000-01-05,0.084167,0.085958,0.082974,0.084526,188352000
2000-01-06,0.078675,0.084167,0.075452,0.084167,120480000
2000-01-07,0.079989,0.080824,0.077124,0.078318,71184000


In [10]:
engine.add_instrument(NVDA_STOCKS_INSTRUMENT)

In [11]:
from nautilus_trader.persistence.wranglers import BarDataWrangler

wrangler = BarDataWrangler(
    NVDA_STOCKS_1DAY_BARTYPE,
    NVDA_STOCKS_INSTRUMENT
)

nvda_1day_bars_list: list[Bar] = wrangler.process(ydata_df)



In [12]:
engine.add_data(nvda_1day_bars_list)


In [13]:
config = MACDStrategyConfig(instrument_id=NVDA_STOCKS_INSTRUMENT.id, bar_type_1day=NVDA_STOCKS_1DAY_BARTYPE)
strategy = MACDStrategy(config=config)
engine.add_strategy(strategy)


In [14]:
# Execute the backtest
engine.run()



ValueError: `total` amount was negative

In [ ]:
# Clean up resources
engine.dispose()

In [15]:
engine.trader.generate_orders_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20000112-000000-001-001-1,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-001,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-009,MARKET,SELL,10000,...,None,None,None,None,None,None,None,863a3c95-c961-4501-ab68-c16779e3da13,947635200000000000,947635200000000000
O-20000113-000000-001-001-2,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-002,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-011,MARKET,SELL,10000,...,None,None,None,None,None,None,None,54432f3a-0c8d-4933-ba1c-4fdaa42bac14,947721600000000000,947721600000000000
O-20000114-000000-001-001-3,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-003,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-013,MARKET,SELL,10000,...,None,None,None,None,None,None,None,485b31fe-2ebe-4e21-8218-f2a1cedf27ab,947808000000000000,947808000000000000
O-20000118-000000-001-001-4,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-004,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-015,MARKET,BUY,10000,...,None,None,None,None,None,None,None,2d4da12b-8a90-490a-a7ac-d5d044e8158c,948153600000000000,948153600000000000
O-20000119-000000-001-001-5,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-005,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-017,MARKET,BUY,10000,...,None,None,None,None,None,None,None,963e7a55-5514-4322-bb9f-1622e92ddab5,948240000000000000,948240000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20100812-000000-001-001-2627,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-5275,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-7944,MARKET,SELL,10000,...,None,None,None,None,None,None,None,4a3f50cb-099c-453e-9427-40635dffd2f1,1281571200000000000,1281571200000000000
O-20100813-000000-001-000-2649,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-1-5276,NVDA.NASDAQ-MACDStrategy-000,NASDAQ-001,NASDAQ-1-7946,MARKET,SELL,10000,...,None,None,None,None,None,None,None,34f334eb-f30a-49e4-ae56-5aca213c54b7,1281657600000000000,1281657600000000000
O-20100813-000000-001-001-2628,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-1-5277,NVDA.NASDAQ-MACDStrategy-001,NASDAQ-001,NASDAQ-1-7947,MARKET,SELL,10000,...,None,None,None,None,None,None,None,41754cb7-bef6-4d4e-aa80-793c1844a0d4,1281657600000000000,1281657600000000000


In [16]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,
NVDA.NASDAQ-MACDStrategy-001-3c5f9747-f257-411b-b161-aae1bb8f9ba6,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20000112-000000-001-001-1,O-20000120-000000-001-001-6,SELL,FLAT,0,30000,2000-01-12 00:00:00+00:00,948326400000000000,2000-01-20 00:00:00+00:00,6.912000e+14,0.08,0.09,['0.00 USD'],-0.125,-300.00 USD
NVDA.NASDAQ-MACDStrategy-001-a6adc4d0-e4c0-43d4-886a-29972622f3ea,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20000121-000000-001-001-7,O-20000126-000000-001-001-10,BUY,FLAT,0,20000,2000-01-21 00:00:00+00:00,948844800000000000,2000-01-26 00:00:00+00:00,4.320000e+14,0.09,0.08,['0.00 USD'],-0.11111,-200.00 USD
NVDA.NASDAQ-MACDStrategy-001-7866b58a-7a38-465e-88b4-62a6905a09f4,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20000127-000000-001-001-11,O-20000216-000000-001-001-24,SELL,FLAT,0,60000,2000-01-27 00:00:00+00:00,950659200000000000,2000-02-16 00:00:00+00:00,1.728000e+15,0.07277777777777779,0.09428571428571429,['0.00 USD'],-0.29553,-1499.99 USD
NVDA.NASDAQ-MACDStrategy-000-46bde48c-bf70-40f5-8a3c-457d7c7f0082,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20000131-000000-001-000-1,O-20000224-000000-001-000-18,SELL,FLAT,0,90000,2000-01-31 00:00:00+00:00,951350400000000000,2000-02-24 00:00:00+00:00,2.073600e+15,0.07555555555555556,0.11222222222222222,['0.00 USD'],-0.48529,-3299.96 USD
NVDA.NASDAQ-MACDStrategy-001-edebc096-a151-4f8e-adc1-3e9ed2b2c14f,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20000217-000000-001-001-25,O-20020814-000000-001-001-644,BUY,FLAT,0,1000000,2000-02-17 00:00:00+00:00,1029283200000000000,2002-08-14 00:00:00+00:00,7.853760e+16,0.30674494331829727,0.252548387096774,['0.00 USD'],-0.17668,-125100.06 USD
NVDA.NASDAQ-MACDStrategy-000-690bf760-4cec-4460-b5ae-8676447896bc,BACKTEST-NVDA1DAY-001,MACDStrategy-000,NVDA.NASDAQ,NASDAQ-001,O-20000225-000000-001-000-19,O-20021002-000000-001-000-670,BUY,FLAT,0,1590000,2000-02-25 00:00:00+00:00,1033516800000000000,2002-10-02 00:00:00+00:00,8.208000e+16,0.3523588091748106,0.22739263803681,['0.00 USD'],-0.35466,-235199.73 USD
NVDA.NASDAQ-MACDStrategy-001-16648cae-d4fc-415b-b61c-af6c79969b94,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20020815-000000-001-001-645,O-20020820-000000-001-001-648,SELL,FLAT,0,20000,2002-08-15 00:00:00+00:00,1029801600000000000,2002-08-20 00:00:00+00:00,4.320000e+14,0.075,0.095,['0.00 USD'],-0.26667,-400.00 USD
NVDA.NASDAQ-MACDStrategy-001-6481e839-ffa1-4b66-b842-0dbbd403ec75,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20020821-000000-001-001-649,O-20020904-000000-001-001-658,BUY,FLAT,0,50000,2002-08-21 00:00:00+00:00,1031097600000000000,2002-09-04 00:00:00+00:00,1.209600e+15,0.092,0.076,['0.00 USD'],-0.17391,-800.00 USD
NVDA.NASDAQ-MACDStrategy-001-99c27a35-2533-4016-b776-9d62f4c13468,BACKTEST-NVDA1DAY-001,MACDStrategy-001,NVDA.NASDAQ,NASDAQ-001,O-20020905-000000-001-001-659,O-20021108-000000-001-001-704,SELL,FLAT,0,190000,2002-09-05 00:00:00+00:00,1036713600000000000,2002-11-08 00:00:00+00:00,5.529600e+15,0.06747368421052631,0.08652173913043479,['0.00 USD'],-0.2823,-4199.94 USD
